In [1]:
import sys
print(sys.executable)

c:\Users\dangn\anaconda3\envs\tensorflowgpu\python.exe


In [2]:
import tensorflow as tf   
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\dangn\Downloads\covid19vaccinesbycountybydemographic.csv")

In [3]:
df

,county,county_type,demographic_category,demographic_value,est_population,est_age_12plus_pop,est_age_5plus_pop,administered_date,partially_vaccinated,total_partially_vaccinated,...,cumulative_fully_vaccinated,at_least_one_dose,cumulative_at_least_one_dose,cumulative_unvax_total_pop,cumulative_unvax_12plus_pop,cumulative_unvax_5plus_pop,suppress_data,up_to_date_count,cumulative_up_to_date_count,administered_year
0,NaN,HPI_RCP_TRACT,HPI Quartile Census Mixed,1,NaN,NaN,NaN,2024-08-23,0.0,17088.0,...,69129.0,0.0,86225.0,NaN,NaN,NaN,False,0,8609,2024
1,NaN,HPI_RCP_TRACT,HPI Quartile Census Mixed,1,NaN,NaN,NaN,2024-08-22,0.0,17088.0,...,69129.0,0.0,86225.0,NaN,NaN,NaN,False,0,8609,2024
2,NaN,HPI_RCP_TRACT,HPI Quartile Census Mixed,1,NaN,NaN,NaN,2024-08-21,4.0,17088.0,...,69129.0,4.0,86225.0,NaN,NaN,NaN,False,8,8609,2024
3,NaN,HPI_RCP_TRACT,HPI Quartile Census Mixed,1,NaN,NaN,NaN,2024-08-20,0.0,17084.0,...,69129.0,0.0,86221.0,NaN,NaN,NaN,False,0,8601,2024
4,NaN,HPI_RCP_TRACT,HPI Quartile Census Mixed,1,NaN,NaN,NaN,2024-08-19,2.0,17084.0,...,69129.0,2.0,86221.0,NaN,NaN,NaN,False,2,8601,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185832,Yuba,DERIVED FROM RECIP ZIP,VEM Quartile,4,39.0,28.0,39.0,2020-07-31,0.0,0.0,...,0.0,0.0,0.0,39.0,28.0,39.0,False,0,0,2020
2185833,Yuba,DERIVED FROM RECIP ZIP,VEM Quartile,4,39.0,28.0,39.0,2020-07-30,0.0,0.0,...,0.0,0.0,0.0,39.0,28.0,39.0,False,0,0,2020
2185834,Yuba,DERIVED FROM RECIP ZIP,VEM Quartile,4,39.0,28.0,39.0,2020-07-29,0.0,0.0,...,0.0,0.0,0.0,39.0,28.0,39.0,False,0,0,2020
2185835,Yuba,DERIVED FROM RECIP ZIP,VEM Quartile,4,39.0,28.0,39.0,2020-07-28,0.0,0.0,...,0.0,0.0,0.0,39.0,28.0,39.0,False,0,0,2020


In [4]:
df['demographic_category'].unique()

array(['HPI Quartile Census Mixed', 'Age Group', 'Gender',
       'Race/Ethnicity', 'VEM Quartile'], dtype=object)

In [6]:
df = df[df['demographic_category'].str.contains("Race/Ethnicity")]

df['demographic_category'].unique()

array(['Race/Ethnicity'], dtype=object)

In [7]:
df['demographic_value'].unique()

array(['American Indian or Alaska Native', 'Asian',
       'Black or African American', 'Latino', 'Multiracial',
       'Native Hawaiian or Other Pacific Islander', 'Other Race',
       'Unknown', 'White'], dtype=object)

In [9]:
df = df[~df['demographic_value'].str.contains("Other Race|Unknown")]

df['demographic_value'].unique()


array(['American Indian or Alaska Native', 'Asian',
       'Black or African American', 'Latino', 'Multiracial',
       'Native Hawaiian or Other Pacific Islander', 'White'], dtype=object)

In [13]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

encoder = LabelEncoder()
df['county_encoded'] = encoder.fit_transform(df['county'])
df['demographic_encoded'] = encoder.fit_transform(df['demographic_value'])

In [16]:
df['total_population'] = df['est_population'] + df['est_age_12plus_pop'] + df['est_age_5plus_pop']

df['percent_fully'] = df['cumulative_fully_vaccinated'] / df['total_population']

In [18]:
df['administered_date'] = pd.to_datetime(df['administered_date'], format = '%Y-%m-%d')

df['time_since_start'] = (df['administered_date'] - df['administered_date'].min()).dt.days

In [20]:
prep_df = df[['percent_fully', 'county_encoded', 'demographic_encoded', 'time_since_start']]

prep_df

,percent_fully,county_encoded,demographic_encoded,time_since_start
24240,0.360569,0,0,1488
24241,0.360569,0,0,1487
24242,0.360569,0,0,1486
24243,0.360569,0,0,1485
24244,0.360569,0,0,1484
...,...,...,...,...
2179876,0.000000,58,6,4
2179877,0.000000,58,6,3
2179878,0.000000,58,6,2
2179879,0.000000,58,6,1


In [33]:
prep_df = prep_df[prep_df['percent_fully'] != 0]

prep_df = prep_df[prep_df['percent_fully'].notnull()]

prep_df

,percent_fully,county_encoded,demographic_encoded,time_since_start
24240,0.360569,0,0,1488
24241,0.360569,0,0,1487
24242,0.360569,0,0,1486
24243,0.360569,0,0,1485
24244,0.360569,0,0,1484
...,...,...,...,...
2179837,0.000009,58,6,43
2179838,0.000009,58,6,42
2179839,0.000009,58,6,41
2179840,0.000009,58,6,40


In [34]:
time_step = 10

grouped_county = prep_df.groupby(['county_encoded', 'demographic_encoded'])

In [35]:
X = []
y = []

for name, group in grouped_county:
    values = group[['percent_fully', 'county_encoded', 'demographic_encoded', 'time_since_start']].values

    for i in range(len(values) - time_step):
        X.append(values[i:i+time_step,:])
        y.append(values[i+time_step, 0])

X = np.array(X)
y = np.array(y)

In [36]:
X.shape

(518314, 10, 4)

In [37]:
np.set_printoptions(suppress=True)

X

array([[[   0.360569  ,    0.        ,    0.        , 1488.        ],
        [   0.360569  ,    0.        ,    0.        , 1487.        ],
        [   0.360569  ,    0.        ,    0.        , 1486.        ],
        ...,
        [   0.360569  ,    0.        ,    0.        , 1481.        ],
        [   0.360569  ,    0.        ,    0.        , 1480.        ],
        [   0.360569  ,    0.        ,    0.        , 1479.        ]],

       [[   0.360569  ,    0.        ,    0.        , 1487.        ],
        [   0.360569  ,    0.        ,    0.        , 1486.        ],
        [   0.360569  ,    0.        ,    0.        , 1485.        ],
        ...,
        [   0.360569  ,    0.        ,    0.        , 1480.        ],
        [   0.360569  ,    0.        ,    0.        , 1479.        ],
        [   0.360569  ,    0.        ,    0.        , 1478.        ]],

       [[   0.360569  ,    0.        ,    0.        , 1486.        ],
        [   0.360569  ,    0.        ,    0.        , 1485. 

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [39]:
model = Sequential()

model.add(LSTM(20, activation = "relu", input_shape = (X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 20)                2000      
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,021
Trainable params: 2,021
Non-trainable params: 0
_________________________________________________________________


In [40]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [41]:
history = model.fit(X, y, epochs = 100, batch_size=32, validation_split=0.2)

Epoch 1/100
12958/12958 [==============================] - 234s 18ms/step - loss: 0.0090 - val_loss: 1.5491e-04
Epoch 2/100
12958/12958 [==============================] - 231s 18ms/step - loss: 3.3751e-05 - val_loss: 2.6441e-05
Epoch 3/100
12958/12958 [==============================] - 234s 18ms/step - loss: 2.8506e-05 - val_loss: 9.6981e-07
Epoch 4/100
12958/12958 [==============================] - 228s 18ms/step - loss: 2.9237e-05 - val_loss: 1.4367e-06
Epoch 5/100
12958/12958 [==============================] - 232s 18ms/step - loss: 2.9399e-05 - val_loss: 2.3050e-06
Epoch 6/100
12958/12958 [==============================] - 229s 18ms/step - loss: 2.2327e-05 - val_loss: 5.0265e-06
Epoch 7/100
12958/12958 [==============================] - 225s 17ms/step - loss: 2.3190e-05 - val_loss: 9.0440e-07
Epoch 8/100
12958/12958 [==============================] - 239s 18ms/step - loss: 1.8519e-05 - val_loss: 5.3453e-07
Epoch 9/100
12958/12958 [==============================] - 239s 18ms/step - 

In [43]:
import os
print(os.getcwd())

c:\Users\dangn\AppData\Local\Programs\Microsoft VS Code


In [45]:
model.save('my_model.h5')

ImportError: `save_model()` using h5 format requires h5py. Could not import h5py.

In [48]:
model.save_weights('my_model_weights_checkpoint')